In [1]:
# Standard imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import logomaker

# Insert path to mavenn beginning of path
import os
import sys
abs_path_to_mavenn = os.path.abspath('../../../')
sys.path.insert(0, abs_path_to_mavenn)

# Load mavenn
import mavenn
print(mavenn.__path__)

['/Users/tareen/Desktop/Research_Projects/2020_mavenn_github/mavenn/mavenn']


In [2]:
# Load example data
data_df = mavenn.load_example_dataset('sortseq')
print(f'training N: {len(data_df)}')
data_df.head()

training N: 45778


,x,training_set,ct_0,ct_1,ct_2,ct_3,ct_4,ct_5,ct_6,ct_7,ct_8,ct_9
0,AAATACACACTTGCTGCTTCCGGCTCGTATGTTGTGTGG,True,0,0,0,1,0,0,0,0,0,0
1,AAATTTACACTGTATGCTTCCGGCTCGCATGGCGTTTGC,False,0,0,1,0,0,0,0,0,0,0
2,AAATTTACACTTTATGCATCAGACTCGTATGTTGTGTGG,True,1,0,0,0,0,0,0,0,0,0
3,AAATTTACACTTTATGCTTCTGGCGCGTATGCGGCGTGG,False,0,0,0,1,0,0,0,0,0,0
4,AACATTACATTTTATGCTTCCGGCTCGTATGGTGTGTGG,False,0,1,0,0,0,0,0,0,0,0


In [3]:
# Comptue sequence length and number of bins
L = len(data_df['x'][0])
Y = len(data_df.filter(regex='ct_*').columns)
print(f'L={L}, Y={Y}')

# Split into training and test data
ix = data_df['training_set']
training_df = data_df[ix].copy()
training_df.reset_index(inplace=True)
test_df = data_df[~ix].copy()
test_df.reset_index(inplace=True)
print(f'N training: {len(training_df)}')
print(f'N test: {len(test_df)}')

L=39, Y=10
N training: 36517
N test: 9261


In [4]:
x = training_df['x'].values
y = training_df.filter(regex='ct_*').values

In [5]:
# Set seeed
mavenn.set_seed(0)

# Define model
model = mavenn.Model(regression_type='MPA',
                     L=L,
                     Y=Y,
                     alphabet='dna',
                     gpmap_type='additive')

# set data
model.set_data(x=x,
               y=y)

N = 36,517 observations set as training data.
Data shuffled.
Time to set data: 0.368 sec.


In [6]:
# Fit model to data
history = model.fit(learning_rate=.005,
                    epochs=1000,
                    early_stopping=True,
                    early_stopping_patience=10)

 
LSMR            Least-squares solution of  Ax = b

The matrix A has    36517 rows  and      156 cols
damp = 0.00000000000000e+00

atol = 1.00e-06                 conlim = 1.00e+08

btol = 1.00e-06             maxiter =      156

 
   itn      x(1)       norm r    norm Ar  compatible   LS      norm A   cond A
     0  0.00000e+00  1.003e+03  8.855e+05   1.0e+00  8.8e-01
     1  5.77073e-03  5.463e+02  1.704e+04   5.4e-01  3.0e-02  1.1e+03  1.0e+00
     2  2.75827e-02  4.918e+02  4.297e+03   4.9e-01  8.3e-03  1.1e+03  1.5e+01
     3 -2.72657e-02  4.840e+02  2.201e+03   4.8e-01  4.3e-03  1.1e+03  2.2e+01
     4 -9.15828e-02  4.813e+02  7.809e+02   4.8e-01  1.5e-03  1.1e+03  2.2e+01
     5 -9.14609e-02  4.811e+02  5.160e+02   4.8e-01  1.0e-03  1.1e+03  2.0e+01
     6 -9.13989e-02  4.811e+02  3.653e+02   4.8e-01  5.6e-04  1.4e+03  2.0e+01
     7 -8.74901e-02  4.810e+02  1.222e+02   4.8e-01  1.7e-04  1.5e+03  2.5e+01
     8 -8.78064e-02  4.810e+02  5.669e+01   4.8e-01  7.9e-05  1.5e+03  2.3

In [7]:
# Save model
model.save('sortseq_mpa_additive')

Model saved to these files:
	sortseq_mpa_additive.pickle
	sortseq_mpa_additive.h5


In [8]:
# Load model
model = mavenn.load('sortseq_mpa_additive')

Model loaded from these files:
	sortseq_mpa_additive.pickle
	sortseq_mpa_additive.h5


In [9]:
# Get G-P map parameters
theta_dict = model.get_theta(gauge='uniform')

# Get additive parameters suitable for logomaker
theta_logomaker_df = theta_dict['logomaker_df']

# Create grid in phi space
phi_lim = [-5, 3]
phi_grid = np.linspace(phi_lim[0], phi_lim[1], 1000)

# Create array of allowable y values
Y = model.model.Y    # Y = number of bins
y_lim = [-.5, Y-.5]
y_all = range(Y)

# Compute matrix of p(y|phi) values
measurement_process = model.p_of_y_given_phi(y_all, phi_grid)

# Create figure with two panels
fig, axs = plt.subplots(1,2,figsize=[12,4])

# Left panel: draw logo using logomaker
ax = axs[0]
logo = logomaker.Logo(theta_logomaker_df, ax=ax)
ax.set_ylabel(r'parameter value ($\theta_{l:c}$)')
ax.set_xlabel(r'position ($l$)')
ax.set_title('G-P map parameters')

# Right panel: draw measurement process as heatmap
ax = axs[1]
im = ax.imshow(measurement_process,
               cmap='Greens',
               extent=phi_lim+y_lim,
               vmin=0,
               origin='lower',
               interpolation='nearest',
               aspect="auto")
ax.set_yticks(y_all)
ax.set_ylabel('bin number (y)')
ax.set_xlabel('latent phenotype ($\phi$)')
ax.set_title('measurement process')
cb = plt.colorbar(im)
cb.set_label('probability  $p(y|\phi)$', rotation=-90, va="bottom")

# Fix up plot
fig.tight_layout(w_pad=3)
fig.savefig('sortseq_mpa_visualization.png')
plt.show()